In [3]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime


In [4]:
# Reading in tickers from csv file
tickers = pd.read_csv('Tickers_Example.csv')
ticker_lst = ['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BB.TO',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']
#for i in range(len(tickers.index)):
    #ticker_lst.append(tickers.iloc[i,0])

ticker_lst

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BB.TO',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

In [5]:
# Retrieving data from S&P500 
start_date = '2021-01-01'
end_date = '2024-11-02'

sp500_ticker = yf.Ticker('^GSPC')
sp500 = sp500_ticker.history(start=start_date, end=end_date, interval='1mo')[['Close']]
sp500.index = sp500.index.strftime('%Y-%m-%d')
sp500_returns = sp500.ffill().pct_change().dropna()
sp500_returns.rename(columns={'Close': 'S&P500 Returns'}, inplace=True)
sp500_returns.head()

,S&P500 Returns
Date,
2021-02-01,0.026091
2021-03-01,0.042439
2021-04-01,0.052425
2021-05-01,0.005486
2021-06-01,0.022214


In [6]:
# Getting tsx historical data
tsx_ticker = yf.Ticker('XIU.TO')
tsx = tsx_ticker.history(start=start_date,end=end_date, interval='1mo')[['Close']]
tsx.index = tsx.index.strftime('%Y-%m-%d')
#tsx.head()
tsx_returns = tsx.ffill().pct_change().dropna()
tsx_returns.rename(columns={'Close': 'TSX60 Returns'}, inplace=True)
tsx_returns.head()

,TSX60 Returns
Date,
2021-02-01,0.036511
2021-03-01,0.052981
2021-04-01,0.023049
2021-05-01,0.030849
2021-06-01,0.033423


In [7]:
# combining dataframe to get total pct change 
sp500tsx60 = sp500_returns.join(tsx_returns)
sp500tsx60['Total % Change'] = sp500tsx60.mean(axis=1)
sp500tsx60.head()

,S&P500 Returns,TSX60 Returns,Total % Change
Date,,,
2021-02-01,0.026091,0.036511,0.031301
2021-03-01,0.042439,0.052981,0.047710
2021-04-01,0.052425,0.023049,0.037737
2021-05-01,0.005486,0.030849,0.018168
2021-06-01,0.022214,0.033423,0.027818


In [8]:
# Create function that takes our list of stocks and assigns each one a tracking error based on the average of s&p 500 and tsx 60
# The function also finds the deviation of the stock returns with the market returns

t_error = []
magnitude = []
def tracking_error (tickerlst, benchmarkdf, start_date, end_date):
    for i in range(len(tickerlst)):
        stock = yf.Ticker(tickerlst[i])
        stock_data = stock.history(start=start_date, end=end_date, interval='1mo')[['Close']]
        stock_data.index = stock_data.index.strftime('%Y-%m-%d')
        # calculate returns for stock
        stock_returns = stock_data.ffill().pct_change().dropna()
        stock_and_benchmark = stock_returns.join(benchmarkdf['Total % Change'])
        # calculate difference in return of stock and benchmark
        difference = stock_and_benchmark['Close'] - stock_and_benchmark['Total % Change']
        magnitude.append(abs(difference.mean()))
        t_error.append(difference.std())

    df = pd.DataFrame({
        'Ticker': tickerlst,
        'Tracking Error': t_error,
        'Return Difference': magnitude
    })

    return df

error_return_df = tracking_error(ticker_lst, sp500tsx60, start_date, end_date)
error_return_df

,Ticker,Tracking Error,Return Difference
0,AAPL,0.056286,0.003643
1,ABBV,0.061605,0.008706
2,ABT,0.040735,0.009583
3,ACN,0.053850,0.001180
4,AIG,0.063995,0.008713
5,AMZN,0.078306,0.001329
6,AXP,0.056025,0.012514
7,BA,0.095034,0.010688
8,BAC,0.064277,0.003547
9,BB.TO,0.135808,0.037437


In [15]:
# Creating a function that sorts the stocks into 'ratings' based on their error and returns
# A lower tracking error and lower deviation of returns should be prioritized
# Sorts it from best to worst rating
def stock_rating(df, weight_track, weight_return):
    rating = []
    
    numstocks = len(df)
    for i in range(numstocks):
        rating.append(df.iloc[i,1]*weight_track + df.iloc[i,2]*weight_return)

    # Sort stocks from highest to lowest rating
    df['Rating'] = rating
    df = df.sort_values(by='Rating', ascending=False).reset_index(drop=True)
    
    return df

# Testing the ratings when  weighting tracking_error is weighted 0.1 and return difference is weighted 0.9
first = stock_rating(error_return_df, 0.1, 0.9)
# Testing the ratings when the tracking_error is weighted 0.3 and return difference is weighted 0.7
second = stock_rating(error_return_df, 0.3, 0.7)
# Testing the ratings when the tracking_error is weighted 0.5 and return difference is weighted 0.5
third = stock_rating(error_return_df, 0.5, 0.5)
# Testing the ratings when the tracking_error is weighted 0.7 and return difference is weighted 0.3
fourth = stock_rating(error_return_df, 0.7, 0.3)
# Testing the ratings when the weighting tracking_error is weighted 0.9 and return difference is weighted 0.1
fifth = stock_rating(error_return_df, 0.9, 0.1)

first

,Ticker,Tracking Error,Return Difference,Rating
0,BB.TO,0.902685,1.000000,0.990268
1,PYPL,0.535366,0.704520,0.687605
2,LLY,0.448600,0.630511,0.612320
3,BIIB,0.539331,0.477963,0.484099
4,CAT,0.334103,0.312879,0.315001
5,AXP,0.146193,0.329016,0.310734
6,PFE,0.310029,0.307659,0.307896
7,BA,0.516070,0.279860,0.303481
8,T.TO,0.072626,0.256417,0.238038
9,BMY,0.237926,0.237047,0.237135


In [16]:
# Creating a function that sorts the stocks into 'ratings' based on their error and returns
# A lower tracking error and lower deviation of returns should be prioritized
# Sorts it from best to worst rating
def stock_rating(df, weight_track, weight_return):
    rating = []
    
    numstocks = len(df)
    for i in range(numstocks):
        df['Tracking Error'] = (df['Tracking Error'] - df['Tracking Error'].min()) / (df['Tracking Error'].max() - df['Tracking Error'].min())
        df['Return Difference'] = (df['Return Difference'] - df['Return Difference'].min()) / (df['Return Difference'].max() - df['Return Difference'].min())
        df['Rating'] = weight_track * df['Tracking Error'] + weight_return * df['Return Difference']

    # Sort stocks from highest to lowest rating
    stocks_by_rating = df.sort_values(by='Rating')
    
    return stocks_by_rating

first = stock_rating(error_return_df, 0.1, 0.9)
# Testing the ratings when the tracking_error is weighted 0.3 and return difference is weighted 0.7
second = stock_rating(error_return_df, 0.3, 0.7)
# Testing the ratings when the tracking_error is weighted 0.5 and return difference is weighted 0.5
third = stock_rating(error_return_df, 0.5, 0.5)
# Testing the ratings when the tracking_error is weighted 0.7 and return difference is weighted 0.3
fourth = stock_rating(error_return_df, 0.7, 0.3)
# Testing the ratings when the weighting tracking_error is weighted 0.9 and return difference is weighted 0.1
fifth = stock_rating(error_return_df, 0.9, 0.1)

fifth

,Ticker,Tracking Error,Return Difference,Rating
28,RY.TO,0.000000,0.110888,0.011089
22,PEP,0.009942,0.098407,0.018788
2,ABT,0.001213,0.250092,0.026101
17,KO,0.052567,0.032790,0.050589
31,TD.TO,0.049775,0.109614,0.055758
32,TXN,0.092225,0.004301,0.083432
16,CL,0.087432,0.113854,0.090074
30,T.TO,0.072626,0.256417,0.091005
34,UNP,0.096825,0.070014,0.094144
24,PG,0.109916,0.048903,0.103815


In [27]:
# Creating portfolios using the top 12 stocks from each dataframe
# Assuming equal weighting among stocks 

def create_portfolio (investment, df, numstocks):
    # Dividing the initial investment equally among all the stocks
    investment_per_stock = investment / numstocks
    
    portfolio = pd.DataFrame()

    for i in range(numstocks):
        ticker = yf.Ticker(df.iloc[i,0])
        stock_prices = ticker.history(start=start_date,end=end_date, interval='1mo')[['Close']] # Retrieving close prices for specific stock
        stock_prices.index = stock_prices.index.strftime('%Y-%m-%d') # Getting rid of timestamps
        #Dropping null values
        stock_prices = stock_prices.dropna()
        shares = investment_per_stock / stock_prices.iloc[i,0]  # Calculating initial amount of shares bought for each stock
        portfolio[df.iloc[i,0]] = stock_prices['Close'] * shares

    # Finding total portfolio value
    portfolio['Total Portfolio Value']=portfolio.sum(axis=1)

    #Change to datetime index to use resample function
    portfolio.index = pd.to_datetime(stock_prices.index)

    #Converting the price to percent returns
    portfolio = portfolio.ffill().pct_change().dropna()

    return portfolio

portfolio1 = create_portfolio(1000000, first, 15)
print('The average percent returns for the first portoflio', portfolio1['Total Portfolio Value'].mean())

portfolio2 = create_portfolio(1000000, second, 15)
print('The average percent returns for the second portoflio', portfolio2['Total Portfolio Value'].mean())

portfolio3 = create_portfolio(1000000, third, 15)
print('The average percent returns for the third portoflio', portfolio3['Total Portfolio Value'].mean())

portfolio4 = create_portfolio(1000000, fourth, 15)
print('The average percent returns for the fourth portoflio', portfolio4['Total Portfolio Value'].mean())

portfolio5 = create_portfolio(1000000, fifth, 15)
print('The average percent returns for the fifth portoflio', portfolio5['Total Portfolio Value'].mean())

print('The return of s&p500 and tsx60 was', sp500tsx60['Total % Change'].mean())

The average percent returns for the first portoflio 0.009470641131138973
The average percent returns for the second portoflio 0.00921836058372143
The average percent returns for the third portoflio 0.00933300890255714
The average percent returns for the fourth portoflio 0.00951006578561138
The average percent returns for the fifth portoflio 0.009442011803973435
The return of s&p500 and tsx60 was 0.011331926019406
